In [ ]:
import numpy as np
import os
import subprocess
import glob
from pathlib import Path 

In [ ]:
video_list = ["/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-27e78d56-1026.13.28.06/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-48d51866-1026.12.46.58/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-55255d89-1026.13.08.31/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-cff3a4f0-1026.18.33.28/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-5e06f1b4-1026.13.44.59/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-69937a1e-1027.19.15.09/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-7b7d9b3c-1026.15.05.53/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-c2276efc-1026.14.08.03/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-d23ac540-1026.19.38.53/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-d4a620c3-1026.12.17.12/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-9447355f-1026.16.57.36/ren.mp4", "/home/kennychufk/workspace/pythonWs/test-run-al-outside/truth/rltruth-1313ed22-1026.19.16.09/ren.mp4"]

In [ ]:
video_list = sorted(video_list)

In [ ]:
selected_video_list = video_list

In [ ]:
selected_video_list

In [ ]:
# add framenum
def overlay_frame_num(filename, width, height):
    tokens = ["ffmpeg"]
    tokens+=["-i", filename]
    tokens+=["-vf"]
    tokens+=[f"scale={width}:{height},drawtext=fontfile=Arial.ttf:"+" text=%{n}: x=(w-tw)/2: y=h-(2*lh): fontcolor=white:fontsize=72: box=1: boxcolor=0x00000099"]
    tokens+=[str(Path(filename).with_suffix(''))+'_frame_num.mp4']
    return run_process(tokens)

In [ ]:
def trim_video(filename):
    with open(Path(filename).parent.joinpath('frame.txt')) as f:
        start_frame = int(f.read())
    end_frame = start_frame+ 20 * 60
    tokens = ["ffmpeg"]
    tokens+=["-i", filename]
    tokens+=["-vf"]
    tokens+=[f"select=between(n\\,{start_frame}\\,{end_frame}),setpts=PTS-STARTPTS"]
    tokens+=["-y"]
    tokens+=[str(Path(filename).with_suffix(''))+'-trimmed.mp4']
    return run_process(tokens)

In [ ]:
for filename in selected_video_list:
    trim_video(filename)

In [ ]:
mosaic_video_list = [str(Path(filename).with_suffix(''))+'-trimmed.mp4' for filename in selected_video_list]
mosaic_video_list

In [ ]:
def run_process(tokens):
    process_popen = subprocess.Popen(tokens)
    try:
        process_popen.wait(timeout=360);
    except subprocess.TimeoutExpired:
        os.kill(process_popen.pid, signal.SIGTERM)
    code = process_popen.returncode
    return code

In [ ]:
for video_path in video_list:
    overlay_frame_num(video_path, 480, -1)

In [ ]:
##### get start frame num ##########

def get_start_frame_nums(paths):
    result = []
    for filename in paths:
        with open(Path(filename).parent.joinpath('frame.txt')) as f:
            start_frame_num = int(f.read())
            result+=[start_frame_num]
    return result

In [ ]:
def get_coordinate(r,c):
    widths = []
    heights = []
    for i in range(c):
        widths+=["w"+str(i)]
    for i in range(r):
        heights+=["h"+str(i)]
    x = ("+".join(widths)) if len(widths)>0 else "0"
    y = ("+".join(heights)) if len(heights)>0 else "0"
    return "{}_{}".format(x,y)
def get_abs_coordinate(r,c, single_width, single_height):
    return "{}_{}".format(c * single_width, r * single_height)
# def gen_xstack_layout(row, col):
#     size = col * row
#     coordinates = []
#     for r in range(row):
#         for c in range(col):
#             coordinates+=[get_coordinate(r,c)]
#     return "inputs="+str(size)+":layout="+("|".join(coordinates))

In [ ]:
def get_spiral_layout(vid_count, row, col, single_width, single_height):
    size = col * row
    coordinates = []
    c, r = col//2, row//2
    counter = 0
    filled_counter = 0
    same_dir_extent=0
    same_dir_travelled = 0
    change_dir_counter = -1
    sign=1
    while (filled_counter<size and filled_counter < vid_count):
#         print("r={}, c={}".format(r,c))
        if (c<col and r<row and c>=0 and r>=0):
            coordinates+=[get_abs_coordinate(r,c, single_width, single_height)]
            filled_counter+=1
        if (same_dir_extent==same_dir_travelled):
#             print("change direction")
            change_dir_counter+=1
            same_dir_travelled = 0
            if (change_dir_counter%2 ==0):
                same_dir_extent+=1
            if (change_dir_counter%4 == 1 or change_dir_counter%4 == 3):
                sign*=-1
        if (change_dir_counter%2==0):
            c+=sign
        else:
            r+=sign
        same_dir_travelled += 1
        counter+=1
    filter_str = ""
    for i in range(filled_counter):
        filter_str+=f"[{i}:v]scale=480:-1[scaled{i}];"
    for i in range(filled_counter):
        filter_str+="[scaled{}]".format(i)
    filter_str+="xstack=inputs="+str(filled_counter)
    filter_str+=":layout="+("|".join(coordinates))+"[v]"
    return "{}".format(filter_str)

In [ ]:
def generate_mosaic(prefix, row, col, single_width, single_height, video_paths):
    tokens = ["ffmpeg"]
    input_tokens = []
    size = min(row*col, len(video_paths))
    for i in range(len(video_paths)):
        input_tokens+=["-i", video_paths[i]]
    tokens+=input_tokens
    tokens+=["-t"]
    tokens+=["20"]
    tokens+=["-filter_complex"]
    tokens+=[get_spiral_layout(size, row, col, single_width, single_height)]
    tokens+=["-map"]
    tokens+=["[v]"]
    tokens+=["{}_{}by{}.mp4".format(prefix, row, col)]
    print(tokens)
    print()
    print(" ".join(tokens))
    ffmpeg_popen = subprocess.Popen(tokens)
    try:
        ffmpeg_popen.wait(timeout=360);
    except subprocess.TimeoutExpired:
        os.kill(ffmpeg_popen.pid, signal.SIGTERM)
    code = ffmpeg_popen.returncode
    return code

In [ ]:
generate_mosaic('truthall', 3, 4, 480, 360, video_list)